# Processing eROSITA data

## Import Statements

In [2]:
from daxa.mission import eRASS1DE
from daxa.archive import Archive
from daxa.process.simple import full_process_erosita

## An Archive to be processed

Every processing function implemented in DAXA takes an Archive instance as its first argument; if you don't already know what that is then you should go back and read the following tutorials:

* [Creating a DAXA archive](archives.html) - This explains how to create an archive, load an existing archive, and the various properties and features of DAXA archives.
* [Using DAXA missions](missions.html) - Here we explain what DAXA mission classes are and how to use them to select only the data you need.

Here we create an archive of eRASS DR1 observations of the galaxy cluster Abell 3266:

In [5]:
er = eRASS1DE()
er.filter_on_name('A3266')
arch = Archive('Abell3266', er)

## One-line solution

Though we provide individual functions that wrap the various steps required to reduce and prepare eROSITA data, and they can be used separately for greater control over the configuration parameters, we also include a one-line solution which executes the processing steps with default configuration.

We believe that the default parameters are adequate for most use cases, and this allows for users unfamiliar with the intricacies of eROSITA data to easily start working with it. Executing the following will automatically generate cleaned combined event lists for all telescope modules selected upon the original declaration of the mission.

In [6]:
full_process_erosita(arch)

eRASS DE:1 - Finding flares in observations: 100%|██████████████████████████████| 2/2 [00:00<00:00,  3.28it/s]


TypeError: The lo_en and hi_en arguments must be astropy quantities in units that can be converted to keV.

## General arguments for all processing functions

There are some arguments that all processing functions will take - they don't control the behaviour of the processing step itself, but instead how the commands are executed:

* **num_cores** - The number of cores that the function is allowed to use. This is set globally by the daxa.NUM_CORES constant (if set before any other DAXA function is imported), and by default is 90% of the cores available on the system.
* **timeout** - This controls how long an individual instance of the process is allowed to run before it is cancelled; the whole function may run for longer depending how many pieces of data need processing and how many cores are allocated. The default is generally null, but it can be set using an astropy quantity with time units.

## Breaking down the eROSITA processing steps

[flaregti function](../../daxa.process.xmm.html#daxa.process.erosita.clean.flaregti)

[cleaned_evt_lists function](../../daxa.process.xmm.html#daxa.process.erosita.assemble.cleaned_evt_lists)